In [1]:
# 🔐 API segura via Colab
import os
from google.colab import userdata
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
GEMINI_URL = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?key={os.environ['GOOGLE_API_KEY']}"

    👋 Olá! Sou o Chef.IA — seu assistente de cozinha inteligente.

    🍽️ Até 4 ingredientes por receita (sal, pimenta e azeite são curingas).
    🧂 Falo com clareza, sem termos técnicos.
    🔄 Reaproveito sobras com ideias criativas.
    🍳 Muitos ingredientes? Divido em grupos e crio receitas diferentes.
    ⏳ Se houver validade, priorizo o uso.
    Me diga o que tem aí ou digite 'sair' para encerrar

In [9]:
# 🤖 Função de resposta e regras aplicadas
import requests

def gerar_prompt(texto_usuario):
    return (
    "Você é um chef prático, direto e simpático, com linguagem acessível e foco em economia e aproveitamento.\n"
    "Regras importantes:\n"
    "- Use no máximo 4 ingredientes por receita. Sal, pimenta-do-reino e azeite são curingas e não contam.\n"
    "- Nunca use todos os ingredientes se houver mais de 5; divida e crie até 3 receitas simples e distintas.\n"
    "- Fale com clareza. Evite termos técnicos ou linguagem infantilizada.\n"
    "- Utilize medidas práticas: colheres, xícaras, unidades, gramas, kg, ml ou litros — escolha conforme o alimento.\n"
    "- Dê dicas rápidas de conservação (ex: como armazenar, congelar ou reaproveitar).\n"
    "- Sempre que possível, sugira uso integral (cascas, talos, sobras) ou alternativas econômicas.\n"
    "- Se algum ingrediente tiver validade informada, mencione quanto tempo falta e priorize seu uso.\n"
    "- Evite exageros ou receitas muito elaboradas. O foco é praticidade com o que já se tem.\n"
    "- Responda de forma organizada, com nome da receita, ingredientes e instruções simples.\n"
    "- Verifique a data de validade do produto se for menor que à data atual, não gere receita e sugira o descarte.\n"
    "- Seja gentil, mas objetivo. O usuário busca praticidade e ideias realistas para o dia a dia.\n"
    f"\nIngredientes fornecidos: {texto_usuario}\n"
    "Responda com simpatia, clareza e foco na utilidade."

    )

def responder_chefia(prompt):
    payload = {"contents": [{"parts": [{"text": prompt}]}]}
    try:
        r = requests.post(GEMINI_URL, json=payload)
        r.raise_for_status()
        return r.json()['candidates'][0]['content']['parts'][0]['text'].strip()
    except:
        return "Desculpe, houve um erro ao consultar a IA."

In [5]:
# 📆 Funções: extrair validade e gerar link para Google Calendar
from urllib.parse import quote
from datetime import datetime
import re

def gerar_link_google_calendar(nome_alimento, data_validade, hora="08:00"):
    try:
        data = datetime.strptime(data_validade, "%Y-%m-%d")
        inicio = data.strftime("%Y%m%dT") + hora.replace(":", "") + "00"
        fim = data.strftime("%Y%m%dT") + str(int(hora[:2]) + 1).zfill(2) + hora[2:] + "00"
        titulo = f"Usar {nome_alimento} antes de vencer"
        return (
            f"https://calendar.google.com/calendar/render?action=TEMPLATE"
            f"&text={quote(titulo)}&dates={inicio}/{fim}"
            f"&details={quote('Lembrete criado com ChefIA')}"
        )
    except Exception as e:
        return None

def extrair_validade(texto):
    match = re.search(r'(\b\w+).*?venc[ea]?.*?(\d{1,2}[/-]\d{1,2}[/-]?\d{0,4})', texto.lower())
    if match:
        alimento = match.group(1)
        data = match.group(2).replace("-", "/")
        partes = data.split("/")
        if len(partes) != 3:
            return None, None
        if len(partes[2]) == 2:
            partes[2] = "20" + partes[2]
        try:
            data_formatada = f"{partes[2]}-{int(partes[1]):02d}-{int(partes[0]):02d}"
            return alimento, data_formatada
        except:
            return None, None
    return None, None

In [8]:
# 💬 Bloco único de entrada com botão encerrar e suporte a validade + receita
import ipywidgets as widgets
from IPython.display import clear_output, Markdown, display

def iniciar_conversa():
    entrada = widgets.Text(
        placeholder='Ex: batata e cenoura / frango vence 25/05/2025',
        description='Você:',
        layout=widgets.Layout(width='100%')
    )
    botao_enviar = widgets.Button(description='Enviar', button_style='success')
    botao_sair = widgets.Button(description='Encerrar', button_style='danger')
    caixa = widgets.Output()

    def ao_enviar(_):
        texto = entrada.value.strip()
        entrada.value = ''
        clear_output(wait=True)
        display(widgets.VBox([entrada, widgets.HBox([botao_enviar, botao_sair])]))
        display(caixa)
        if texto:
            display(Markdown(f"### 🧑 Você:\n> {texto}"))
            prompt = gerar_prompt(texto)
            resposta = responder_chefia(prompt)
            display(Markdown(f"### 🤖 ChefIA:\n{resposta}"))
            alimento, data = extrair_validade(texto)
            if alimento and data:
                link = gerar_link_google_calendar(alimento, data)
                if link:
                    display(Markdown(f"📅 [Agendar no Google Calendar para usar {alimento}]({link})"))

    def encerrar_conversa(_):
        clear_output(wait=True)
        display(Markdown("🛑 **Conversa encerrada. Obrigado por usar o ChefIA!**"))

    botao_enviar.on_click(ao_enviar)
    botao_sair.on_click(encerrar_conversa)
    display(widgets.VBox([entrada, widgets.HBox([botao_enviar, botao_sair])]))
    display(caixa)

# Inicia automaticamente a interface
iniciar_conversa()

Output()

### 🧑 Você:
> arroz venceuem 26/12/2024

### 🤖 ChefIA:
Olá! Que bom te ver por aqui! Com esse arroz que vence em 26/12/2024, podemos fazer algo bem gostoso e prático. Vamos lá?

**Nome da Receita: Arroz Refogado Básico**

**Ingredientes:**

*   1 xícara de arroz
*   2 xícaras de água

**Instruções:**

1.  Lave bem o arroz em água corrente.
2.  Em uma panela, coloque o arroz e a água.
3.  Adicione sal a gosto.
4.  Leve ao fogo alto até ferver.
5.  Abaixe o fogo e cozinhe com a panela semi-tampada até a água secar (cerca de 15-20 minutos).
6.  Desligue o fogo e deixe a panela tampada por 5 minutos antes de servir.

**Dica Rápida:**

*   Para um sabor extra, refogue o arroz com um pouco de azeite antes de adicionar a água.

**Conservação:**

*   Guarde o arroz cozido na geladeira por até 3 dias em um recipiente fechado.

Espero que goste! 😊